In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 8

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v05"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v05 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V08/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V08/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D14 UTC 12:36


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//122784_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//172497_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115801_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111011_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//232144_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//131902_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//142625_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//153021_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//145102_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//181976_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//223940_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//157807_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//155562_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//127443_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//127450_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//122810_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132930_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//131908_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//142634_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//162410_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//155571_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111018_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//172525_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//181993_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//153028_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115809_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//145109_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//232160_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//122845_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//158142_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111044_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//142643_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132160_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//155578_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//172542_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//153035_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//127468_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//162445_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//116007_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//185000_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//232240_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//225111_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117103_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//232302_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132223_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//145403_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//122853_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//158150_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//127486_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//153043_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//172549_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//142652_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//162462_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//211016_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//155616_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132956_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//225160_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//211033_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//172557_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//155633_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//142659_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//145500_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//162469_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132241_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//132973_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//211034_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//158218_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//153051_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117202_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//232401_V08.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914506_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911102_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812102_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822476_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742998_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//723030_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//726017_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//782688_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//831959_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//652309_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//712800_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//753306_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//642465_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742430_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//662907_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742447_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//652406_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//642472_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//832103_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//782696_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812109_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914704_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//772345_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822494_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//723074_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//713223_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//743008_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911213_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742482_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//642479_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//832300_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//772370_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812207_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//652503_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//664023_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//782704_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822512_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911240_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//753502_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//726078_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//751205_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914805_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//751402_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//772406_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//726086_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//782803_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812306_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911269_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//753610_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822520_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//832428_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//723118_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//664043_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742491_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//751403_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911270_V08.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812472_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742521_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911402_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//753630_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822550_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//832450_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//782950_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//772608_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//812473_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911403_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//742522_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//822560_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//753640_V08.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//832460_V08.csv
/volumes/data/Y2018M

In [14]:
#!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111011_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111012_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111013_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111014_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111015_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111016_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111019_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111020_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//111017_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112130_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112140_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112150_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112160_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112120_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112180_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112190_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112400_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//112300_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114441_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114442_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114443_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114444_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114445_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114446_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114447_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114448_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//114449_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115422_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115423_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115424_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115425_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115426_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115427_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115428_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115429_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//115430_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117101_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117102_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117103_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117104_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117105_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117106_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117107_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117108_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//117109_V08.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//852090_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861001_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861002_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861003_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861004_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861005_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861006_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//862010_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//861007_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911216_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911217_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911218_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911219_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911220_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911230_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911240_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911250_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911261_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911809_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911911_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911912_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911913_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911914_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911915_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911916_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911917_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//911918_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//912808_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//912809_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//912900_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913000_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913100_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913210_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913220_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913230_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913240_V08.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913905_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913906_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913907_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//913908_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914100_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914201_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914202_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914203_V08.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V08//914204_V08.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:31:40.728932


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932

